In [14]:
import trackhhl.toy.simple_generator as toy
import trackhhl.event_model.q_event_model as em
import numpy as np
import itertools
import copy
from dwave.samplers import SimulatedAnnealingSampler
import dimod
import psutil
import time
from scipy.sparse import lil_matrix, csc_matrix, block_diag




In [15]:
N_MODULES = 5
LX = float("+inf")
LY = float("+inf")
Z_SPACING = 1.0

detector = toy.SimpleDetectorGeometry(
    module_id=list(range(N_MODULES)),
    lx=[LX] * N_MODULES,
    ly=[LY] * N_MODULES,
    z=[i + Z_SPACING for i in range(N_MODULES)]
)

generator = toy.SimpleGenerator(
    detector_geometry=detector,
    theta_max=np.pi / 6
)

N_PARTICLES = 2
event = generator.generate_event(N_PARTICLES)
event.hits

[Hit(hit_id=0, x=0.04307604246182619, y=-0.005110414922683672, z=1.0, module_id=0, track_id=0),
 Hit(hit_id=5, x=0.07605354361067525, y=0.5072265088512558, z=1.0, module_id=0, track_id=1),
 Hit(hit_id=1, x=0.08615208492365238, y=-0.010220829845367345, z=2.0, module_id=1, track_id=0),
 Hit(hit_id=6, x=0.1521070872213505, y=1.0144530177025115, z=2.0, module_id=1, track_id=1),
 Hit(hit_id=2, x=0.1292281273854786, y=-0.01533124476805102, z=3.0, module_id=2, track_id=0),
 Hit(hit_id=7, x=0.2281606308320257, y=1.521679526553767, z=3.0, module_id=2, track_id=1),
 Hit(hit_id=3, x=0.17230416984730476, y=-0.02044165969073469, z=4.0, module_id=3, track_id=0),
 Hit(hit_id=8, x=0.304214174442701, y=2.028906035405023, z=4.0, module_id=3, track_id=1),
 Hit(hit_id=4, x=0.215380212309131, y=-0.025552074613418366, z=5.0, module_id=4, track_id=0),
 Hit(hit_id=9, x=0.38026771805337617, y=2.536132544256278, z=5.0, module_id=4, track_id=1)]

In [16]:
params = {
    'alpha': 1.0,
    'beta': 1.0,
    'lambda': 100.0,} 

def generate_hamiltonian_optimized(event, params):
    lambda_val = params.get('lambda')
    alpha = params.get('alpha')
    beta = params.get('beta')

    modules = sorted(event.modules, key=lambda module: module.z)

    
    segments = [
        em.segment(from_hit, to_hit)
        for idx in range(len(modules) - 1)
        for from_hit, to_hit in itertools.product(modules[idx].hits, modules[idx + 1].hits)
    ]

    N = len(segments)  

    #initialize sparse block matrices for effviciency
    A_ang_blocks = []
    A_bif_blocks = []
    A_inh_blocks = []
    b = np.zeros(N)

    #total Hamiltonian into smaller blocks->> better for memory
    block_size = 500  
    num_blocks = (N + block_size - 1) // block_size  

    for block_idx in range(num_blocks):
        start_idx = block_idx * block_size
        end_idx = min(start_idx + block_size, N)

        #lil_matrix for each block
        A_ang_block = lil_matrix((end_idx - start_idx, end_idx - start_idx), dtype=np.float32)
        A_bif_block = lil_matrix((end_idx - start_idx, end_idx - start_idx), dtype=np.float32)
        A_inh_block = lil_matrix((end_idx - start_idx, end_idx - start_idx), dtype=np.float32)

        #filling of papricas
        for i in range(start_idx, end_idx):
            seg_i = segments[i]
            vect_i = seg_i.to_vect()
            norm_i = np.linalg.norm(vect_i)

            for j in range(i + 1, end_idx):  #
                seg_j = segments[j]
                vect_j = seg_j.to_vect()
                norm_j = np.linalg.norm(vect_j)

                cosine = np.dot(vect_i, vect_j) / (norm_i * norm_j)
                if np.abs(cosine - 1) < 1e-9:
                    A_ang_block[i - start_idx, j - start_idx] = 1
                    A_ang_block[j - start_idx, i - start_idx] = 1  # Symmetry with positive sign

                if seg_i.from_hit == seg_j.from_hit and seg_i.to_hit != seg_j.to_hit:
                    A_bif_block[i - start_idx, j - start_idx] = -alpha
                    A_bif_block[j - start_idx, i - start_idx] = -alpha  # Symmetry with negative sign

                if seg_i.from_hit != seg_j.from_hit and seg_i.to_hit == seg_j.to_hit:
                    A_bif_block[i - start_idx, j - start_idx] = -alpha
                    A_bif_block[j - start_idx, i - start_idx] = -alpha  # Symmetry with negative sign

                s_ab = int(seg_i.from_hit.module_id == 1 and seg_j.to_hit.module_id == 1)
                if s_ab > 0:
                    A_inh_block[i - start_idx, j - start_idx] = beta * s_ab * s_ab
                    A_inh_block[j - start_idx, i - start_idx] = beta * s_ab * s_ab  # Symmetry with positive sign

        A_ang_blocks.append(A_ang_block)
        A_bif_blocks.append(A_bif_block)
        A_inh_blocks.append(A_inh_block)

    # combine withblock diagonal
    A_ang = block_diag(A_ang_blocks, format='csc')
    A_bif = block_diag(A_bif_blocks, format='csc')
    A_inh = block_diag(A_inh_blocks, format='csc')

    A = -1 * (A_ang + A_bif + A_inh)

    return A, b, segments

#performance measurement
process = psutil.Process()

start_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
start_time = time.time()

A, b, segments = generate_hamiltonian_optimized(event, params)
end_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
end_time = time.time()

memory_used = end_memory - start_memory
time_taken = end_time - start_time

print(f"Memory used: {memory_used:.2f} MB")
print(f"Time taken: {time_taken:.6f} seconds")

Memory used: 0.00 MB
Time taken: 0.011631 seconds


In [17]:
import dimod
import psutil
import time
from scipy.sparse import csc_matrix

def qubosolver(A, b):

    #performance measurement
    process = psutil.Process()
    start_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
    start_time = time.time()

    #Keep A sparse
    A = csc_matrix(A)

    bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)

    #vectors for efficiency 
    bqm.add_variables_from({i: b[i] for i in range(len(b))})

    row, col = A.nonzero()  
    for i, j in zip(row, col):
        if i != j:  
            bqm.add_interaction(i, j, A[i, j])

    sampler = SimulatedAnnealingSampler()
    response = sampler.sample(bqm, num_reads=100)

    best_sample = response.first.sample
    sol_sample = np.fromiter(best_sample.values(), dtype=int)  

    end_memory = process.memory_info().rss / (1024 ** 2) 
    end_time = time.time()

   
    memory_used = end_memory - start_memory
    time_taken = end_time - start_time

    print(f"Solution:{sol_sample}")
    print(f"Memory {memory_used:.2f} MB")
    print(f"Time {time_taken:.6f} seconds")

    return sol_sample

qubosolver(A, b)


Solution:[1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1]
Memory 0.00 MB
Time 0.055139 seconds


array([1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1])